In [1]:
!pip install -q sentence-transformers pymilvus langchain-community


In [2]:
from pymilvus import MilvusClient, DataType

CLUSTER_ENDPOINT = "https://in03-22706440ab83439.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "fd3df03a845da1e42df2a2975f1d4733f62b044415d021f9f6047ef1f631504509c1a870e1370ebc8587c04883fae0c72c886204"

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

# Define the collection name
collection_name = "document_embeddings"

# Connect to Milvus (ensure you've set up the connection correctly)
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)

In [4]:
# Load your local embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-4-388e7c865f49>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
# load the documents from Milvus
vector_store = Milvus(
    collection_name=collection_name,
    embedding_function=embeddings,
    connection_args={
        "uri": CLUSTER_ENDPOINT,
        "token": TOKEN,  # omit if unused
    }
)

<ipython-input-5-4e7d03429f39>:2: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vector_store = Milvus(


In [6]:
query = "What are the symptoms of asthma?"
retrieved_docs = vector_store.similarity_search(query, k=3)

print("Top Retrieved Documents:")
for doc in retrieved_docs:
    # print the source
    print(f"Source: {doc.metadata['source']}")
    # print the first 300 characters of the page content
    print(doc.page_content[:300])  # Print a snippet
    print("\n")


Top Retrieved Documents:
Source: ../docs\3.pdf
Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that they are having to work
harder to breathe or are not getting enough oxygen, indicating respiratory
dist


Source: ../docs\2.pdf
Global Regions
Donate
Home / Health topics / Chronic respiratory diseases
CreditsChronic respiratory diseases
Chronic respiratory diseases (CRDs) a ect the airways and other structures of the lungs. Some of the most
common are chronic obstructive pulmonary disease (COPD), asthma, occupational lung d


Source: ../docs\4.pdf
For Everyone 
Situation Summary
About Respiratory Illnesses
Risk Factors
Prevention
Treatment
Data and Trends
Resources to Prepare for Flu, COVID-19, and RSV
Respiratory Virus Guidance
Health Care Providers 
Best Practices for Patient Care
Talking with Patients
  Sign up for Email Updates  
ESPA




In [7]:
%pip install -q  langchain_openai ctransformers sacremoses


In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# ✅ Install required packages (if not already installed)
!pip install transformers accelerate bitsandbytes

In [10]:
!pip install -U bitsandbytes # Upgrade bitsandbytes to the latest version

In [11]:
# ✅ Imports
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# ✅ Device setup (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Model name
model_name = "microsoft/phi-2"

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Load model in 8-bit precision with automatic device placement
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,       # 8-bit quantization to save RAM
    device_map="auto",       # Automatically map to GPU if available
    torch_dtype="auto"       # Automatically choose best dtype
)

# ✅ Set up the text-generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Using device: cuda


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
# Provide a specific prompt
prompt = "What are the symptoms of asthma?"

# Generate text with proper truncation and max length
response = generator(prompt,
                     max_length=200,  # Increase max length
                     num_return_sequences=1,
                     truncation=True,  # Ensure truncation
                     do_sample=True,   # Enable sampling for variety
                     top_p=0.9,        # Control diversity
                     temperature=0.7)  # Control randomness

print("Generated Response:", response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response: What are the symptoms of asthma?
What are the risk factors of asthma?
How is asthma diagnosed?
How is asthma treated?
What is the prognosis of asthma?
What is the best way to prevent asthma?
What is the latest research on asthma?
Asthma is a chronic inflammatory disease that affects the airways (bronchi) and causes recurrent symptoms such as wheezing, coughing, chest tightness, and shortness of breath. Asthma is a leading cause of hospitalization among children.
Asthma is a chronic inflammatory disease that affects the airways (bronchi) and causes recurrent symptoms such as wheezing, coughing, chest tightness, and shortness of breath. Asthma is a leading cause of hospitalization among children. Asthma can be life-threatening.
Asthma is a chronic inflammatory disease that affects the airways (bronchi) and causes recurrent symptoms such as wheezing, coughing, chest tight


In [15]:
# Ensure pad_token is set if not already
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding if not set

# Extract the content of the top retrieved documents, limiting the total length
retrieved_content = " ".join([doc.page_content for doc in retrieved_docs])
# Limit the retrieved content to a specific number of characters
max_retrieved_content_length = 200  # Adjust as needed
retrieved_content = retrieved_content[:max_retrieved_content_length]

# Update the prompt by appending the retrieved content
contextual_prompt = f"Context: {retrieved_content}\n\nQuestion: {prompt}\nAnswer:"

# Tokenize the input, further reducing max_length to avoid exceeding model's limit
encoded_input = tokenizer.encode(contextual_prompt, truncation=True, max_length=250, padding="max_length", return_tensors="pt")  # Reduced max_length to 250

# Check the tokenized input length
input_length = len(encoded_input[0])

# Calculate remaining space for generated tokens = max - input_length
print("maximum length: "+str(tokenizer.model_max_length))
print("input length: "+str(input_length))
remaining_tokens = tokenizer.model_max_length - input_length

# Ensure that the generation does not exceed the model's token limit
response = generator(contextual_prompt,
                     max_new_tokens=min(remaining_tokens, 150),  # Limit the number of generated tokens
                     num_return_sequences=1,
                     do_sample=True,   # Enable sampling for variety
                     top_p=0.9,        # Control diversity
                     temperature=0.7,  # Control randomness
                     pad_token_id=tokenizer.eos_token_id)  # Explicitly use eos_token_id for padding
# Display the generated response
print("Generated Response:", response[0]['generated_text'])

maximum length: 2048
input length: 250
Generated Response: Context: Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that the

Question: What are the symptoms of asthma?
Answer: Symptoms of asthma include wheezing, shortness of breath, chest tightness, and coughing.

Question: What is the main cause of asthma?
Answer: The main cause of asthma is the narrowing of the airways due to inflammation and the production of excess mucus.

Question: How can asthma be managed?
Answer: Asthma can be managed through medications such as inhalers, avoiding triggers, and maintaining a healthy lifestyle.

Question: What are the risk factors for developing asthma?
Answer: Risk factors for developing asthma include a family history of the condition, exposure to allergens or irritants, and respiratory infections.

Question: What are the signs of respiratory distres